In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

from glob import glob
from tqdm import tqdm

In [ ]:
def test_load_image(test_file, target_size=(256,256)):
    img = cv2.imread(test_file, cv2.IMREAD_GRAYSCALE)
    img = img / 255
    img = cv2.resize(img, target_size)
    img = np.reshape(img, img.shape + (1,))
    img = np.reshape(img,(1,) + img.shape)
    return img

def test_generator(test_files, target_size=(256,256)):
    for test_file in test_files:
        yield test_load_image(test_file, target_size)
        
def save_result(save_path, npyfile, test_files):
    for i, item in enumerate(npyfile):
        result_file = test_files[i]
        img = (item[:, :, 0] * 255.).astype(np.uint8)

        filename, fileext = os.path.splitext(os.path.basename(result_file))

        result_file = os.path.join(save_path, "%s_predict%s" % (filename, fileext))

        cv2.imwrite(result_file, img)

In [ ]:
def rv_3ch(og_img_path, mask_path, size):
    
    #module import
    import cv2
    
    # im_read
    og_img = cv2.imread(og_img_path)
    mask = cv2.imread(mask_path)
    
    # clahe
    clahe = cv2.createCLAHE(5.0,(12,12))
    
    ## og_img_convert
    cvt_img = cv2.resize(og_img, size)
    cvt_img = cv2.cvtColor(cvt_img,cv2.COLOR_BGR2GRAY)
    cvt_img = clahe.apply(cvt_img)  # 1 channel
    cvt_img = np.repeat(cvt_img[:,:,np.newaxis],3,-1)

    ## mask
    cvt_mask = cv2.resize(mask, size)
    cvt_mask = cv2.cvtColor(cvt_mask, cv2.COLOR_BGR2GRAY)

    ## remove
    remove = cv2.bitwise_and(cvt_img, cvt_img, mask = cvt_mask)
    
    return remove

In [ ]:
import tensorflow
model = tensorflow.keras.models.load_model('/content/drive/MyDrive/3차플젝drive/test/model/unet_2_100.h5',compile=False)

In [ ]:
import cv2
normal_path ='/content/drive/MyDrive/3차플젝drive/공모전_이전_캐글/PD/penumonia/KG_JPG/normal/'
normal_files = glob(os.path.join(normal_path, "*.jpg"))

qnormal_path = '/content/drive/MyDrive/3차플젝drive/공모전_이전_캐글/PD/penumonia/KG_JPG/normal?/'
qnormal_files = glob(os.path.join(qnormal_path, "*.jpg"))

pneu_path = '/content/drive/MyDrive/3차플젝drive/공모전_이전_캐글/PD/penumonia/KG_JPG/pneumonia/'
pneu_files = glob(os.path.join(pneu_path, "*.jpg"))

save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/3차플젝drive/공모전_이전_캐글/PD/total_file.csv',index_col=0)
df['count'] = 1
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['class_cat'] = le.fit_transform(df['class'])
group = df.groupby('id')[['target','class_cat','count']].sum()
group = group.reset_index()

In [ ]:
group = group.sort_values(by='count',ascending=False)
group['class_cat'] = group['class_cat'].replace(0,'Lung Opacity').replace(1,'No Lung Opacity / Not Normal').replace(2,'Normal')

In [ ]:
#normal = 2
#no_lung_opacity = 1
#lung opacity = 0

normal = group[group['class_cat']=='Normal']
normal_list = list(normal['id'])

qnormal = group[group['class_cat']=='No Lung Opacity / Not Normal']
qnormal_list = list(qnormal['id'])

PN = group[group['class_cat']=='Lung Opacity']
PN_list = list(PN['id'])

group.head(1)

,id,target,class_cat,count
11018,76f71a93-8105-4c79-a010-0cfa86f0061a,4,Lung Opacity,4


In [ ]:
print(len(normal_list))
print(len(qnormal_list))
print(len(PN_list))

len(normal_list) + len(qnormal_list) + len(PN_list)

8851
11821
6012


26684

In [ ]:
print(len(normal_files))
print(len(qnormal_files))
print(len(pneu_files))
len(normal_files) + len(qnormal_files) + len(pneu_files)

14477
6195
6011


26683

In [ ]:
pneu_files[0].split('pneumonia/')[1].split('.jpg')[0]

'35e2f247-6930-4b7d-bc43-baaa622cce2c'

In [ ]:
#PN
pneu_path = '/content/drive/MyDrive/3차플젝drive/공모전_이전_캐글/PD/penumonia/KG_JPG/pneumonia/'
pneu_files = glob(os.path.join(pneu_path, "*.jpg"))

save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'


t_gen= test_generator(pneu_files, target_size=(512,512))
B = model.predict_generator(t_gen, len(pneu_files), verbose=1)
save_result(save_dir, B, pneu_files)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  if __name__ == '__main__':


6011/6011 [==============================] - 174s 27ms/step


In [ ]:
#QN
qnormal_path = '/content/drive/MyDrive/3차플젝drive/공모전_이전_캐글/PD/penumonia/KG_JPG/normal?/'
qnormal_files = glob(os.path.join(qnormal_path, "*.jpg"))

save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'


t_gen= test_generator(qnormal_files, target_size=(512,512))
B = model.predict_generator(t_gen, len(qnormal_files), verbose=1)
save_result(save_dir, B, qnormal_files)

   3/6195 [..............................] - ETA: 2:53

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  if __name__ == '__main__':


6195/6195 [==============================] - 197s 32ms/step


In [ ]:
# #NM
# normal_path ='/content/drive/MyDrive/3차플젝drive/공모전_이전_캐글/PD/penumonia/KG_JPG/normal/'
# normal_files = glob(os.path.join(normal_path, "*.jpg"))

# save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'


# # t_gen= test_generator(normal_files, target_size=(512,512))
# # B = model.predict_generator(t_gen, len(normal_files), verbose=1)
# # save_result(save_dir, B, normal_files)

In [ ]:
normal_files_1 = normal_files[:3000]
normal_files_2 = normal_files[3000:6000]
normal_files_3 = normal_files[6000:9000]
normal_files_4 = normal_files[9000:12000]
normal_files_5 = normal_files[12000:]

In [ ]:
len(normal_files_4)

3000

In [ ]:
save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'
t_gen= test_generator(normal_files_1, target_size=(512,512))
B = model.predict_generator(t_gen, len(normal_files_1), verbose=1)
save_result(save_dir, B, normal_files_1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


3000/3000 [==============================] - 871s 290ms/step


In [ ]:
normal_files_2
save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'


t_gen= test_generator(normal_files_2, target_size=(512,512))
B = model.predict_generator(t_gen, len(normal_files_2), verbose=1)
save_result(save_dir, B, normal_files_2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


3000/3000 [==============================] - 851s 284ms/step


In [ ]:
normal_files_3
save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'


t_gen= test_generator(normal_files_3, target_size=(512,512))
B = model.predict_generator(t_gen, len(normal_files_3), verbose=1)
save_result(save_dir, B, normal_files_3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


3000/3000 [==============================] - 847s 283ms/step


In [ ]:
normal_files_4
save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'


t_gen= test_generator(normal_files_4, target_size=(512,512))
B = model.predict_generator(t_gen, len(normal_files_4), verbose=1)
save_result(save_dir, B, normal_files_4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


3000/3000 [==============================] - 872s 291ms/step


In [ ]:
normal_files_5
save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'


t_gen= test_generator(normal_files_5, target_size=(512,512))
B = model.predict_generator(t_gen, len(normal_files_5), verbose=1)
save_result(save_dir, B, normal_files_5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


2477/2477 [==============================] - 662s 268ms/step


In [2]:
from glob import glob
save_dir = '/content/drive/MyDrive/3차플젝drive/KAGGLE_UNET/UNET_MASK'
save_f = glob(save_dir+'/'+"*.jpg")
len(save_f)

26276

In [3]:
26683-26276

407

In [ ]:
contours = [0]
for i in range(len(contours)):
    stri = str(i)
    print(f'c_m_{stri}')
    globals()['x_{}'.format(i)] = contours[i][(f'contours_min_{stri}')]
    # globals()['x_{}'.format(i)] = 'A'

c_m_0


TypeError: ignored

In [ ]:
contours[9]

0

In [ ]:
contours[0][('contours_min_0')[0][1]]

IndexError: ignored

In [ ]:
[('contours_min_%d'.format(stri))[0][1]][0][1]